In [ ]:
# load data from tsp file
import numpy as np

tsp_data = np.loadtxt('Yourpath\\gr17.2085.tsp')#fri26_d.937 gr17.2085
print(tsp_data.shape)

In [ ]:
import math
import queue
from collections import Counter

def initial_solution(length):
    s = np.random.choice(range(length), length, replace=False) #cities in [0,length]
    return s
    
def objectivefunction(s):
    cost = 0
    for i in range(s.shape[0]-1):
        cost = cost + tsp_data[s[i]][s[i+1]]
    
    cost = cost + tsp_data[s[-1]][s[0]]
    return cost

def initial_population(size, length):
    population = [] # set initially to empty
    
    for _ in range(size):
        population.append(initial_solution(length))
        
    return population


def evaluate_population(pop):
    
    cost_pop = []
    
    for index in range(len(pop)):
        cost_pop.append(objectivefunction(pop[index]))
        
    #cost_pop = [objectivefunction(pop[index]) for index in range(len(pop))] # equivalent version
        
    return cost_pop   

def tournament_selection (pop, cost_pop, k): # k is the size of selection
    
    selec_indiv = np.random.choice(range(len(pop)), k, replace=False) # pick randomly K indexes, k < len(pop)
    
    costs_selec_indiv = [cost_pop[selec_indiv[index]] for index in range(len(selec_indiv))]
    
    index_best_indiv = costs_selec_indiv.index(min(costs_selec_indiv)) # minimization problem
    
    return pop[selec_indiv[index_best_indiv]]  # best indiv among k ones picked randomly from pop

def roulettewheel_selection (pop, cost_pop): 
    
    pop_prob = cost_pop/sum(cost_pop) #divide each cost by the sum of the costs to get a prob. Here best individuals have small probs
    pop_prob = 1 - pop_prob # invert probabilities to increase chances of selecting best individuals 
    
    #print(pop_prob)
    
    index = np.random.choice(len(cost_pop), 1, p=pop_prob)[0] # since it returns a list
    
    #print(index)
    
    return pop[index]  # best indiv among k ones picked randomly from pop
    

def replacement_crossover(parent,offspring,cross_pos):
    
    while (True): # left side
        left_repeated_values = [item for item, count in Counter(offspring[:cross_pos[1]+1]).items() if count > 1]
        if (len(left_repeated_values)==0):
            break
        #print(left_repeated_values)
        
        for index in range (len(left_repeated_values)): # replace these vals by new ones from parent_1 according to an order
            val = left_repeated_values[index]
            last_index = np.argwhere(offspring==val)[-1]
            first_index = np.argwhere(offspring==val)[0]
            offspring[first_index] = parent[last_index]
            
            
    while (True): # right side
        right_repeated_values = [item for item, count in Counter(offspring[cross_pos[0]:]).items() if count > 1]
        if (len(right_repeated_values)==0):
            break
        
        for index in range(len(right_repeated_values)): # replace these vals by new ones from parent_1 according to an order
            val = right_repeated_values[index]
            last_index = np.argwhere(offspring==val)[-1]
            first_index = np.argwhere(offspring==val)[0]
            offspring[last_index] = parent[first_index]
        
        
    
    copy = np.zeros(len(parent), dtype=np.uint8)
    copy[:] = offspring[:]
        
    return copy
    
    

def pmx_crossover(parent_1, parent_2):
    # pick 2 random crossver positions
    cross_pos = np.random.choice(range(1,len(parent_1)-1), 2, replace=False)
    cross_pos = np.sort(cross_pos)
    
    #print('p1 {}'.format(parent_1))
    #print('p2 {}'.format(parent_2))
    
    #print('cp {}'.format(cross_pos))
    
    offspring_1 = np.zeros(len(parent_1), dtype=np.uint8)
    offspring_1 [:cross_pos[0]] = parent_1 [0:cross_pos[0]]
    offspring_1 [cross_pos[1]:] = parent_1 [cross_pos[1]:]
    offspring_1 [cross_pos[0]:cross_pos[1]+1] = parent_2 [cross_pos[0]:cross_pos[1]+1]
    final_offspring_1 = replacement_crossover(parent_1,offspring_1,cross_pos)
    
    offspring_2 = np.zeros(len(parent_1), dtype=np.uint8)
    offspring_2 [:cross_pos[0]] = parent_2 [0:cross_pos[0]]
    offspring_2 [cross_pos[1]:] = parent_2 [cross_pos[1]:]
    offspring_2 [cross_pos[0]:cross_pos[1]+1] = parent_1 [cross_pos[0]:cross_pos[1]+1]
    final_offspring_2 = replacement_crossover(parent_2,offspring_2,cross_pos)
    
    #print('o1 {}'.format(final_offspring_1))
    #print('o2 {}'.format(final_offspring_2))
    
    return offspring_1, offspring_2

def mutation(individual):
    copy = np.zeros(len(individual), dtype=np.uint8)
    index = np.random.choice(range(len(individual)), 2, replace=False) # pick randomly 2 indices from individual
    individual[index[0]],individual[index[1]] = individual[index[1]],individual[index[0]] # swap values located at these indices
    copy[:] = individual[:]
    return copy
        
def mutation_decision(prob): 
    flags = ['yes','no']
    decision = np.random.choice(flags, 1, p=[prob, 1-prob]) # select yes or no according to p
    if decision == 'yes':
        return True
    else:
        return False
    
def getbestparentandofs(parent_1,parent_2, ofs_1, ofs_2):
    cost_ofs_1 = objectivefunction(ofs_1)
    cost_ofs_2 = objectivefunction(ofs_2)
    cost_par_1 = objectivefunction(parent_1)
    cost_par_2 = objectivefunction(parent_2)
    
    if (cost_par_1<cost_par_2):
        if(cost_ofs_1<cost_ofs_2):
            return np.copy(parent_1),np.copy(ofs_1)
        else:
            return np.copy(parent_1),np.copy(ofs_2)
    else:
        if(cost_ofs_1<cost_ofs_2):
            return np.copy(parent_2),np.copy(ofs_1)
        else:
            return np.copy(parent_2),np.copy(ofs_2)

In [ ]:
def ga():
    pop_size = 200 # must be a power of 2 for code optimization
    individual_size = 17
    max_generations = 100
    mutation_prob = 0.3
    tournament_size = 8
    elitism_size = 40 # must be a power of 2 for code optimization 
    steady = False

    pop = initial_population(pop_size,individual_size)
    cost_pop = evaluate_population(pop)

    generations = 0
    while (generations < max_generations):
    
        intermediate_pop = []
    
        # prepare elitism
        indices = np.argpartition(cost_pop, elitism_size)[:elitism_size]
        intermediate_pop = [pop[indices[index]] for index in range (elitism_size)] # keep n best individuals
    
        for _ in range ((pop_size - elitism_size)//2):
            # select tow parents
            #parent_1 = roulettewheel_selection(pop, cost_pop) #tournament_selection(pop, cost_pop, tournament_size)
            #parent_2 = roulettewheel_selection(pop, cost_pop)#tournament_selection(pop, cost_pop, tournament_size)
            
            parent_1 = tournament_selection(pop, cost_pop, tournament_size)
            parent_2 = tournament_selection(pop, cost_pop, tournament_size)
        
            # crossover
            ofs_1, ofs_2 = pmx_crossover(parent_1, parent_2)
    
            # mutation with probability
            if (mutation_decision (mutation_prob)):
                ofs_1 = mutation(ofs_1)
            if (mutation_decision (mutation_prob)):
                ofs_2 = mutation(ofs_2)
                
            # steady state replacement  
            if steady:
                ind_1, ind_2 = getbestparentandofs(parent_1,parent_2, ofs_1, ofs_2)
            else:
                ind_1, ind_2 = ofs_1, ofs_2
        
            intermediate_pop.append(ind_1)
            intermediate_pop.append(ind_2)
    
        # replacement of the population
        pop = np.copy(intermediate_pop)
        np.random.shuffle(pop)
    
        # evaluation 
        cost_pop = evaluate_population(pop)
    
        #print(np.min(cost_pop))
    
        generations+=1
    
    index_best = np.array(cost_pop).argmin()

    return cost_pop[index_best], pop[index_best]

In [ ]:
it = 0
while it<1: # you can run multiple sessions
    print (ga())
    it+=1